# Imports

In [1]:
import os
import wandb
import pandas as pd

from pathlib import Path

import project_config as pc

# Retrieve dataset and model

In [2]:
# Init run
run = wandb.init(project=pc.WANDB_PROJECT,
				 entity=pc.WANDB_ENTITY,
				 dir=pc.WANDB_LOCAL_LOGS_PATH,
				 job_type='inference')

# Download latest model version (if not already downloaded)
dataset_artifact = run.use_artifact(f'{pc.DATASET_ARTIFACT_NAME}:latest')
dataset_dir = pc.WANDB_LOCAL_ARTIFACTS_PATH+Path(dataset_artifact._default_root()).stem
if not os.path.exists(dataset_dir):
	_ = dataset_artifact.download(root=dataset_dir)

# Download latest model version (if not already downloaded)
model_artifact = run.use_artifact('model:latest')
model_dir = pc.WANDB_LOCAL_ARTIFACTS_PATH+pc.DATASET_ARTIFACT_NAME+'-'+Path(model_artifact._default_root()).stem
if not os.path.exists(model_dir):
	_ = model_artifact.download(root=model_dir)

# Finish run
run.finish()

wandb: Currently logged in as: facuroffet99 to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


# Inference

In [3]:
from DLOlympus.fastai.inference import FastAIInferencer

# Create inferencer
inferencer = FastAIInferencer(model_dir+'/model.pkl')
inferencer.init()

# Get items
df = pd.read_csv(dataset_dir+'/dataset.csv')
df = df[df['is_valid']]
items = df['file_path'].apply(lambda x: dataset_dir+'/'+x).values

# Get predictions
raw_preds, decoded_preds, outputs = inferencer.process(items)

C:\Users\Facu\Mi unidad\Librerias\DLOlympus\src\DLOlympus\fastai\inference.py:28: UserWarning: List of functions to postprocess outputs not provided, inferring from dataloaders.
  warnings.warn('List of functions to postprocess outputs not provided, inferring from dataloaders.', UserWarning)
C:\Users\Facu\Mi unidad\Librerias\DLOlympus\src\DLOlympus\fastai\inference.py:35: UserWarning: The number of postprocessing functions given or inferred 2 is less than the number of model outputs 3, identity functions added.
  warnings.warn(f'The number of postprocessing functions given or inferred {len(preds_postprocessing_fns)} is less than the number of model outputs {self.n_outputs}, identity functions added.', UserWarning)
c:\Users\Facu\miniconda3\envs\projects\Lib\site-packages\fastai\callback\fp16.py:47: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  self.autocast,self.learn.scaler,self.scales = autocast(dtype=dtype

In [9]:
outputs

((#1451) ['basset_hound','Maine_Coon','american_bulldog','basset_hound','american_bulldog','american_bulldog','english_setter','american_bulldog','Egyptian_Mau','havanese','american_bulldog','samoyed','saint_bernard','samoyed','shiba_inu','keeshond','saint_bernard','american_bulldog','german_shorthaired','samoyed'...],
 (#1451) ['dog','dog','dog','dog','dog','dog','dog','dog','dog','dog','dog','dog','dog','dog','dog','dog','dog','dog','dog','dog'...],
 tensor([[ 64.5689],
         [ 97.3826],
         [174.8780],
         ...,
         [113.7295],
         [ 63.1303],
         [108.8789]]))

In [16]:
from sklearn.metrics import f1_score

# Check metric
f1_score(outputs[0], df['label_breed'].values, average='macro')

0.307743094802659

# ONNX

In [17]:
import albumentations

c, h, w = 3, 64, 64 

stats_mean = [0.485, 0.456, 0.406]
stats_std = [0.229, 0.224, 0.225]

transforms = albumentations.Compose([
	albumentations.SmallestMaxSize(max_size=max(h,w)),
	albumentations.CenterCrop(height=h, width=w),
	albumentations.Normalize(stats_mean, stats_std),
	albumentations.ToTensorV2()
])

In [18]:
# fastai 2 onnx conversion
inferencer.export([c, h, w])

In [19]:
from DLOlympus.onnx.inference import ONNXInferencer

# Create inferencer
inferencer = ONNXInferencer('model.onnx')
inferencer.init(transforms)

onnx_outputs = inferencer.process(items, batch_size=256)

ONNX session created on ['CPUExecutionProvider'].


Processing Batches: 100%|██████████| 6/6 [00:04<00:00,  1.31it/s]
